In [33]:
# ref  https://docs.llamaindex.ai/en/stable/examples/retrievers/recurisve_retriever_nodes_braintrust/

In [1]:
%pip install llama-index-embeddings-together
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install -U llama_hub llama_index braintrust autoevals pypdf pillow transformers torch torchvision
%pip install llama-index-embeddings-huggingface



  Using cached llama_index_embeddings_together-0.2.0-py3-none-any.whl.metadata (660 bytes)
  Using cached llama_index_core-0.11.13.post1-py3-none-any.whl.metadata (2.4 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.35-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.10.6-cp311-cp311-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached pillow-10.4.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached pydantic-2.9.2-py3-none-

In [2]:
!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-postprocessor-rankllm-rerank

  Using cached llama_index_postprocessor_rankllm_rerank-0.2.0-py3-none-any.whl.metadata (2.8 kB)
Using cached llama_index_postprocessor_rankllm_rerank-0.2.0-py3-none-any.whl (4.2 kB)


In [3]:
!pip install rank-llm


  Using cached rank_llm-0.20.2-py3-none-any.whl.metadata (24 kB)
  Using cached pyserini-0.38.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached faiss_cpu-1.8.0.post1-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.7 kB)
  Using cached ftfy-6.2.3-py3-none-any.whl.metadata (7.8 kB)
  Using cached dacite-1.8.1-py3-none-any.whl.metadata (15 kB)
  Using cached fschat-0.2.36-py3-none-any.whl.metadata (20 kB)
  Using cached fastapi-0.115.0-py3-none-any.whl.metadata (27 kB)
  Using cached markdown2-2.5.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached nh3-0.2.18-cp37-abi3-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata (1.7 kB)
  Using cached rich-13.8.1-py3-none-any.whl.metadata (18 kB)
  Using cached shortuuid-1.0.13-py3-none-any.whl.metadata (5.8 kB)
  Using cached uvicorn-0.30.6-py3-none-any.whl.metadata (6.6 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.0-py3-none-any.whl.metadata (13 kB)
  Using cached sen

In [1]:
import sys
import os
import pandas as pd

# Add the parent directory containing both 'EDA' and 'RAG' to sys.path
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Adjust this based on your project structure
sys.path.append(parent_dir)

import pandas as pd
from src_rag.document_loader import DocumentLoader
from src_rag.index_manager import IndexManager
from RAG.src_rag.text_data_handler import TextDataHandler


%load_ext autoreload
%autoreload 2




In [2]:
# %load_ext autoreload
# %autoreload 2
# # NOTE: Replace YOUR_OPENAI_API_KEY with your OpenAI API Key and YOUR_BRAINTRUST_API_KEY with your BrainTrust API key. Do not put it in quotes.
# # Signup for Braintrust at https://braintrustdata.com/ and get your API key at https://www.braintrustdata.com/app/braintrustdata.com/settings/api-keys
# # NOTE: Replace YOUR_OPENAI_KEY with your OpenAI API Key and YOUR_BRAINTRUST_API_KEY with your BrainTrust API key. Do not put it in quotes.
# %env OPENAI_API_KEY=
# #%env BRAINTRUST_API_KEY=
# %env TOKENIZERS_PARALLELISM=true # This is needed to avoid a warning message from Chroma

In [3]:
from pathlib import Path
from llama_index.readers.file import PDFReader
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
import json
from llama_index.core import Document

from llama_index.core.embeddings import resolve_embed_model

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import IndexNode



In [4]:
# Load the Excel file
file_path = "../data/processed_data/df_cluster.xlsx"
df = pd.read_excel(file_path)

# Check if the 'text' column exists
if 'text' not in df.columns:
    raise ValueError("The 'text' column is not present in the provided Excel file.")


# Create Document objects from the 'text' column
docs = [Document(text=row['text']) for _, row in df.iterrows()]

# get API key and create embeddings
from llama_index.embeddings.openai import OpenAIEmbedding

node_parser = SentenceSplitter(chunk_size=1024)

base_nodes = node_parser.get_nodes_from_documents(docs)
# set node ids to be a constant
for idx, node in enumerate(base_nodes):
    node.id_ = f"node-{idx}"

embed_model = OpenAIEmbedding(model="text-embedding-3-large")


#embed_model = resolve_embed_model("local:BAAI/bge-small-en")
llm = OpenAI(model="gpt-3.5-turbo")

base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)

base_retriever = base_index.as_retriever(similarity_top_k=5)


retrievals = base_retriever.retrieve(
    "Can you tell me about the key concepts for keykundra"
)

for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** node-23<br>**Similarity:** 0.31570202848103573<br>**Text:** How do healthcare providers educate patients about the benefits and risks of Keytruda? Healthcare providers offer educational materials, detailed consultations, and regular follow-up to discuss treatment progress and side effects.<br>

**Node ID:** node-21<br>**Similarity:** 0.27950926896466904<br>**Text:** How is awareness being raised for Keytruda as a treatment option? Awareness is being raised through patient advocacy groups, clinical trial publications, and healthcare provider education programs.<br>

**Node ID:** node-20<br>**Similarity:** 0.2780495764018529<br>**Text:** Are there patient support programs available for Keytruda users? Yes, there are several patient support programs that provide financial assistance and counseling services.<br>

**Node ID:** node-22<br>**Similarity:** 0.2750545105911072<br>**Text:** Is there a community support group for patients undergoing treatment with Keytruda? Yes, several online and local support groups are available for sharing experiences and advice.<br>

**Node ID:** node-3<br>**Similarity:** 0.26843246127331855<br>**Text:** What should patients report immediately while on Keytruda treatment? Patients should report any new or worsening symptoms such as cough, chest pain, or changes in vision immediately.<br>

# COn reranker
ref  https://docs.llamaindex.ai/en/stable/examples/node_postprocessor/rankLLM/

In [5]:
# %pip install -q cohere llama-index-postprocessor-cohere-rerank
# %pip install llama-index-postprocessor-rankllm-rerank


In [6]:
index = base_index

In [7]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankllm_rerank import RankLLMRerank

import pandas as pd
import torch
from IPython.display import display, HTML

In [8]:
# with_reranker = True

# query_str = "Can you tell me about the key concepts for keykundra"
# query_bundle = QueryBundle(query_str)
# # configure retriever
# retriever = VectorIndexRetriever(
#     index=index,
#     similarity_top_k=5,
# )
# retrieved_nodes = retriever.retrieve(query_bundle)
# retrieved_nodes.reverse()

# if with_reranker:
#         # configure reranker
#         reranker = RankLLMRerank(
#             model=model, top_n=reranker_top_n, window_size=window_size
#         )
#         retrieved_nodes = reranker.postprocess_nodes(
#             retrieved_nodes, query_bundle
#         )

#         # clear cache, rank_zephyr uses 16GB of GPU VRAM
#         del reranker
#         torch.cuda.empty_cache()

In [9]:
%pip show rank-llm 

Name: rank-llm
Version: 0.20.2
Summary: A Package for running prompt decoders like RankVicuna
Home-page: https://github.com/castorini/rank_llm
Author: 
Author-email: Sahel Sharifymoghaddam <sahel.sharifymoghaddam@uwaterloo.ca>, Ronak Pradeep <rpradeep@uwaterloo.ca>, Jimmy Lin <jimmylin@uwaterloo.ca>
License: 
                              Apache License
                        Version 2.0, January 2004
                     http://www.apache.org/licenses/

TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

1. Definitions.

   "License" shall mean the terms and conditions for use, reproduction,
   and distribution as defined by Sections 1 through 9 of this document.

   "Licensor" shall mean the copyright owner or entity authorized by
   the copyright owner that is granting the License.

   "Legal Entity" shall mean the union of the acting entity and all
   other entities that control, are controlled by, or are under common
   control with that entity. For the purposes of this

In [10]:
# %pip install llama-index-core
# %pip install llama-index-llms-openai
# %pip install llama-index-postprocessor-rankllm-rerank
# %pip install rank-llm


In [11]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankllm_rerank import RankLLMRerank

import pandas as pd
import torch
from IPython.display import display, HTML


def get_retrieved_nodes(
    query_str,
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=False,
    model="rank_zephyr",
    window_size=None,
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)
    retrieved_nodes.reverse()

    if with_reranker:
        # configure reranker
        reranker = RankLLMRerank(
            model=model, top_n=reranker_top_n#, window_size=window_size
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

        # clear cache, rank_zephyr uses 16GB of GPU VRAM
        del reranker
        torch.cuda.empty_cache()

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [12]:
new_nodes = get_retrieved_nodes(
    "Can you tell me about the key concepts for keykundra",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
    model="gpt",
)

ModuleNotFoundError: No module named 'rank_llm.result'

In [17]:
new_nodes = get_retrieved_nodes(
    "Which date did Paul Gauguin arrive in Arles?",
    vector_top_k=50,
    reranker_top_n=3,
    with_reranker=True,
    model="rank_zephyr",
    window_size=15,
)

visualize_retrieved_nodes(new_nodes)

TypeError: __init__() got an unexpected keyword argument 'window_size'